# Calculate a satellite position from Keplerian orbital parameters
The Keplerian motion can be derived from the Newton'a laws, if the bodies are points and there are not any perturbation.
## The Keplerian orbital parameters
Each satellite has six Keplerian parameters:

|name|notation|dimension|variable name in the program|
|-----|-----|-----|-----|
|Semimajor axis of orbital ellipse|$a$|distance|a|
|Numerical eccentricity of ellipse|$e$|-|ecc|
|Epoch of perigee passage|$T_{0}$|time|t0|
|Right ascension of ascending node|$\Omega$|angle|uom|
|Inclination of orbital plane|$i$|angle|i|
|Argument of perigee|$\omega$|angle|low|


The calculation needs and seventh value: the product of G and the earth’s mass: $\mu_{Earth}=m_{Earth}G=3.986004418\cdot10^{14}\frac{m^{3}}{s^{2}}$

In [1]:
# import the required functions and constant from the math module
from math import pi, sqrt, sin, cos, tan, atan, atan2
#and import the numpy module for the matrice calculations
import numpy as np

#Keplerian parameters of the satellite:
#semimajor axis of orbital ellipse:
a=26559821.15
#numerical eccentricity of ellipse:
#generally denoted by e, but this may be means the base of the natural logarithm
ecc=0.0025
#right ascension of ascending node:
#generally denoted by Upper case letter greek OMega
uom=272.8501*(pi/180)
#Inclination of orbital plane:
i=55.054*(pi/180)
#argument of perigee:
#generally denoted by Lower case letter greek OMega
lom=12.354*(pi/180)
#Epoch of perigee passage:
t0=0.0

#The product of G and the earth’s mass:
mu=398600441800000

#The epoch, when the position will be calculated:
et=1000.0

## Calculations in the orbital plane
The firs step is the calculation of the mean angular velocity:

$$n=\sqrt{\frac{\mu}{a^{3}}}$$

$$P=\frac{2\pi}{n}$$



In [2]:
#mean angular satellite velocity:
n=sqrt(mu/a**3)
#revolution period:
period=(2*pi)/n
n, period

(0.00014585830706265586, 43077.32232543011)

The next calculated value is the mean anomaly:

$$M=n\left(t-T_{0}\right)$$

In [3]:
#time from the initial epoch:
t=et-t0
#mean anomaly:
ma=n*t
ma, ma*(180/pi)

(0.14585830706265585, 8.357065401613388)

The eccentric anomaly is calculated by an iteration method:

$$E_{INITIAL}=M$$

$$E_{NEW}=M+e\cdot\sin E_{OLD}$$

In [4]:
#the initial value is the mean anomaly
ea=ma
#the iteration loop
while True:
    old_ea=ea
    ea=ma+ecc*sin(ea)
    if abs(ea-old_ea)<0.0000000000001:
        break
ea, ea*(180/pi)

(0.14622256219750707, 8.377935683506331)

The true anomaly calculated by a complicated formula:

$$\upsilon=2\arctan\left(\sqrt{\frac{1+e}{1-e}}\tan\frac{E}{2}\right)$$

The program has to watch the true return value of the arcus tangent function, which must be similar than the eccentric anomaly.

In [5]:
halfea=ea/2
if (halfea/pi)%1==0.5:
    halfta=halfea
else:
    halfta=atan(sqrt((1+ecc)/(1-ecc))*tan(halfea))
    diff_period=round((halfea-halfta)/pi)
    halfta+=pi*diff_period
ta=2*halfta

#Another method
ta2=2*(atan2(sqrt((1+ecc)/(1-ecc))*sin(ea/2),cos(ea/2))%pi)

ta, ta*(180/pi), ta2, ta2*(180/pi)

(0.14658726891662222,
 8.398831839271692,
 0.14658726891662224,
 8.398831839271693)

If the value of the true anomaly is known, the coordinates in the orbital plane can be calculated:

$$r=a\left(1-e\cdot\cos E\right)=\frac{a\left(1-e^{2}\right)}{1+e\cdot\cos\upsilon}$$

$$x=a\left(\cos E-e\right)$$

$$y=a\left(\sqrt{1-e^{2}}\sin E\right)$$

$$x=r\cdot\cos\upsilon$$

$$y=r\cdot\sin\upsilon$$

$$v_{x}=-\frac{na^{2}}{r}\sin E=-\sqrt{\frac{\mu}{a\left(1-e^{2}\right)}}\sin\upsilon$$

$$v_{x}=\frac{na^{2}}{r}\sqrt{1-e^{2}}\cos E=\sqrt{\frac{\mu}{a\left(1-e^{2}\right)}}\left(\cos\upsilon+e\right)$$

In [6]:
#the distance from the Earth's centre:
r=a*(1-ecc*cos(ea))
#the coordinates in the orbital plane:
ux=a*(cos(ea)-ecc)
uy=a*sqrt(1-ecc**2)*sin(ea)
print(ux, uy)
#check the calculations:
print(r*cos(ta), r*sin(ta))
#calculate the velocity vector in the orbital plane:
velpar=sqrt(mu/(a*(1-ecc**2)))
vx=-velpar*sin(ta)
vy=velpar*(cos(ta)+ecc)
print(vx, vy)

26209988.87600512 3869808.4006395717
26209988.87600512 3869808.400639572
-565.8449823097736 3842.1203530338057


## Calculate the 3D position
The transformation between the orbital plane (the third coordinates is 0) and the 3D space depends from three Keplerian parameters:

$$R=R_{Z}\left(-\Omega\right)\cdot R_{X}\left(-i\right)\cdot R_{Z}\left(-\omega\right)$$

$$R_{Z}\left(-\Omega\right)=\left[\begin{array}{ccc}
\cos\Omega & -\sin\Omega & 0\\
\sin\Omega & \cos\Omega & 0\\
0 & 0 & 1
\end{array}\right]$$

$$R_{X}\left(-i\right)=\left[\begin{array}{ccc}
1 & 0 & 0\\
0 & \cos i & -\sin i\\
0 & \sin i & \cos i
\end{array}\right]$$

$$R_{Z}\left(-\omega\right)=\left[\begin{array}{ccc}
\cos\omega & -\sin\omega & 0\\
\sin\omega & \cos\omega & 0\\
0 & 0 & 1
\end{array}\right]$$

The resultalnt matrix with symbols:

$$R=\left[\begin{array}{ccc}
\cos\varOmega\cos\omega-\sin\varOmega\sin\omega\cos i & -\cos\varOmega\sin\omega-\sin\varOmega\cos\omega\cos i & \sin\varOmega\sin i\\
\sin\varOmega\cos\omega+\cos\varOmega\sin\omega\cos i & -\sin\varOmega\sin\omega+\cos\varOmega\cos\omega\cos i & -\cos\Omega\sin i\\
\sin\omega\sin i & \cos\omega\sin i & \cos i
\end{array}\right]$$

The program calculates by numerical multiplication used the numpy.

In [7]:
#the vectors of position and velocity in NumPy
pos_vec=np.array([ux, uy, 0.0])
vel_vec=np.array([vx, vy, 0.0])

#the rotation matrix of uom in NumPy
rm_uom=np.array([[ cos(uom), sin(uom), 0.0],
                 [-sin(uom), cos(uom), 0.0],
                 [      0.0,      0.0, 1.0]])

#the rotation matrix of i in NumPy
rm_i=np.array([[1.0,     0.0,    0.0],
               [0.0,  cos(i), sin(i)],
               [0.0, -sin(i), cos(i)]])

#the rotation matrix of lom in NumPy
rm_lom=np.array([[ cos(lom), sin(lom), 0.0],
                 [-sin(lom), cos(lom), 0.0],
                 [      0.0,      0.0, 1.0]])

#the resultant rotation matrix
rm=rm_uom @ rm_i @ rm_lom

#the satellite coordinates
pos3d=rm @ pos_vec
#the velocity vector coordinates
vel3d=rm @ vel_vec

print(pos3d)
print(vel3d)
print(rm)


[  2359712.95089947  26346286.81689404   1497951.83143031]
[-2203.03281547   379.29435968 -3175.66552928]
[[ 0.17097225 -0.54821006 -0.81867834]
 [ 0.96954238  0.24150858  0.04075765]
 [ 0.17537409 -0.80071177  0.57280415]]
